# Setup stuff: Connectivity

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, FD1ctCTNk1ARui3xQUkoFyzgLeM3xJ6zgio3r73KowFm, 2>


#  Actual analytics ..

In [5]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


--


In [7]:

#  Getting the airport node id
#

#  l_airport = "MKE"
#  l_airport = "ORD"
#  l_airport = "DEN"
l_airport = "SJC"


l_query  = """
   MATCH (n: Airport) 
   WHERE n.airport_code = {0}
   RETURN ID(n) AS my_id
   LIMIT {0}
   """.format(l_airport)

l_result = my_graph.query(l_query)

print(tabulate(l_result, headers='keys', tablefmt='psql'))






          0/? [?op/s]

OperationError: 6Dcho4mDpksfdgnMjD3FHaBzXf5ty5o5xJAR718nqxx8-2gQdUSS3eEzBVv7fA backtrace (QueryOperation.cpp:67): backtrace (Network.h:331): backtrace (QueryOperation.cpp:70): backtrace (QueryClient.cpp:568): backtrace (OpGraph.cpp:365): backtrace (OpGraph.h:596): backtrace (OpGraph.h:596): backtrace (OpGraph.h:596): backtrace (OpGraph.cpp:367): backtrace (MatchLimitOptimizer.cpp:65): backtrace (MatchLimitOptimizer.cpp:43): backtrace (Evaluate.cpp:2047): backtrace (Evaluate.cpp:2008): backtrace (Evaluate.cpp:1168): column SJC not found (Table.cpp:628): InternalError:ExecutionError: InternalError:ExecutionError: InternalError:ExecutionError

In [ ]:

#  Running analytics.sssp() ..


l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)

# l_weight_property     = None                     #  This errors out
   #
l_weight_property     = "DISTANCE"                 #  Edge property
# l_weight_property     = "NUM_HOPS"

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
   source               = l_source_nodeid,
   # weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


In [ ]:
l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result1.view()
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
# l_result2.view()
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   // RETURN r.type, r.TYPE, r.label, r.start.labels, r.end.labels, r.internal_id, r.start.id, r.end.id, r.DISTANCE, r.NUM_HOPS
   """ )
# l_result.view()
print(tabulate(l_result, headers='keys', tablefmt='psql'))


#  Graph loaded, but before running sssp() 
#     (This is correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge
#  3            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge
#
#  1            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge

#  Graph loaded, after running sssp() 
#     (This is not correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge         #  This data is now wrong
#  1            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge         #  This data is now wrong
#
#  3            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge         #  This data is now wrong


#  Now version 0.5 ..
#
#  Graph loaded, but before running sssp() 
#     (This is correct)
#
#  r.end.id           r.end.labels   r.internal_id    r.label    r.DISTANCE    r.NUM_HOPS    r.TYPE      r.start.id        r.start.labels    r.type
#  562949953421313    ['Airport']    0                FLIES_TO    948          1             FLIES_TO    1                 ['Airport']       edge
#  562949953421312    ['Airport']    2                FLIES_TO    886          1             FLIES_TO    1                 ['Airport'] 	     edge
#  562949953421312    ['Airport']    1                FLIES_TO     66          1             FLIES_TO    0                 ['Airport'] 	     edge
#  562949953421312    ['Airport']    562949953421312  FLIES_TO   1829          1             FLIES_TO    562949953421313   ['Airport'] 	     edge
#  1                  ['Airport']    562949953421314  FLIES_TO    948          1             FLIES_TO    562949953421313   ['Airport'] 	     edge
#  1                  ['Airport']    562949953421313  FLIES_TO    886          1             FLIES_TO    562949953421312   ['Airport'] 	     edge
#  0                  ['Airport']    562949953421315  FLIES_TO     66          1             FLIES_TO    562949953421312   ['Airport'] 	     edge

#  Graph loaded, after running sssp() 
#
#  r.end.id           r.end.labels   r.internal_id    r.label    r.DISTANCE    r.NUM_HOPS    r.TYPE      r.start.id          r.start.labels    r.type
#  2                  ['Airport']    3                FLIES_TO     66          1             FLIES_TO    0                   ['Airport']       edge
#  1                  ['Airport']    2 	              FLIES_TO    948          1             FLIES_TO    3                   ['Airport']       edge
#  2                  ['Airport']    5 	              FLIES_TO   1829          1             FLIES_TO    3                   ['Airport']       edge
#  2                  ['Airport']    4 	              FLIES_TO    886          1             FLIES_TO    1                   ['Airport']       edge
#  3                  ['Airport']    6 	              FLIES_TO    948          1             FLIES_TO    1                   ['Airport']       edge
#  0                  ['Airport']    0 	              FLIES_TO     66          1             FLIES_TO    2                   ['Airport']       edge
#  1                  ['Airport']    1                FLIES_TO    886          1             FLIES_TO    2                   ['Airport']       edge



In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """ ,
   contextualize=True,
   )
l_result.view()


# Running graph.project()



In [ ]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph that analytics.sssp() will not like ..


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.id               = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


In [ ]:
l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result1.view()
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
# l_result2.view()
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


In [ ]:
# ##################################################################

In [ ]:

#  analytics.sssp() now will now excounter at least one edge without a weight
#  property. As such, it generates a (was NaN, now zero) for that path/output.


l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)
l_weight_property     = "DISTANCE"                 #  Edge property
l_max_iterations      = 10000 

#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)

analytics.sssp(
   my_graph,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


In [ ]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result.view()
print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:

#  Running analytics.sssp() should be run against a projection, a sub-graph ..

my_graph2 = my_graph.project(edge_types=["FLIES_TO"])


l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)
l_weight_property     = "DISTANCE"                 #  Edge property
l_max_iterations      = 10000 

#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)

analytics.sssp(
   my_graph2,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))




In [ ]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result.view()
print(tabulate(l_result, headers='keys', tablefmt='psql'))

